In [65]:
#  for installation
# pip,install google-generativeai pinecone-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:

import google.generativeai as genai
from pinecone import ServerlessSpec,Pinecone

In [17]:

pc = Pinecone(api_key='apikey')
genai.configure(api_key="apikey")


# Create a serverless index
index_name = "gemini"
pc.create_index(
    name=index_name,
    dimension=1,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws', 
        region='us-east-1'
    ) 
)   

index = pc.Index(index_name)


In [18]:
# Function to generate embeddings using Google Gemini
def generate_gemini_embedding(text):
    result = genai.embed_content(
        model="models/text-embedding-004",  # Ensure the model returns 768-dimension embeddings
        content=text,
        task_type="retrieval_document",
        title=f"Embedding of: {text[:30]}"
    )
    embedding = result['embedding'][0]  # Extract the embedding
    return embedding

chat_history = [
    "Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today",
    "What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]


vectors = []

for i, message in enumerate(chat_history):
    # Generate the embedding for each chat message
    embedding = generate_gemini_embedding(message)
    
    # Create a unique vector ID for each message
    vector_id = f"message-{i+1}"
    
    # Split message into user and bot components for metadata
    user_message, bot_message = message.split("| Bot: ")
    
    # Append the vector with metadata
    vectors.append({
        "id": vector_id,
        "values": [embedding],  # embedding as a list of floats
        "metadata": {
            "user_message": user_message.strip(),  # Store the user's message
            "bot_message": bot_message.strip(),    # Store the bot's response
            "message_index": i+1                   # Add any additional metadata
        }
    })



# Upsert all vectors including metadata
index.upsert(vectors=vectors,namespace="chat_history")


{'upserted_count': 20}

In [28]:
def search_similar_messages(query, top_k=3):
    # Generate embedding for the query
    query_embedding = generate_gemini_embedding(query)
    
    # Perform the search in the vector database
    # Make sure to request metadata in the response
    response = index.query(
        vector=query_embedding, 
        top_k=top_k, 
        include_metadata=True  # Explicitly include metadata in the response
    )
    return response['matches']
query = " Can you tell what i have to follow to become fit"
similar_messages = search_similar_messages(query, top_k=3)



In [31]:
class TokenLimitExceededError(Exception):
    """Custom exception for when the token limit is exceeded."""
    pass

def combine_bot_messages_and_query(similar_messages, query):
    # Initialize an empty list to store all bot messages
    bot_messages = []
    
    # Collect bot messages from similar messages
    for match in similar_messages:
        bot_message = match.get('metadata', {}).get('bot_message')
        if bot_message:
            bot_messages.append(bot_message)
    
    # Combine all bot messages and the query into one string
    combined_result = " ".join(bot_messages) + " " + query
    
    # Count the tokens in the combined result
    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    
    token_response = model.count_tokens(combined_result) 
    token_count = token_response.total_tokens  # Adjust this line to access the actual token count

    # Raise an exception if the token count exceeds the limit
    if token_count >= 225:
        raise TokenLimitExceededError("Combined result exceeds token limit.")
    
    return combined_result,token_count



try:
    combined_query = combine_bot_messages_and_query(similar_messages, query)
    print("Combined Output:", combined_query[0])
except TokenLimitExceededError as e:
    print(f"Error: {e}")


Combined Output:   Can you tell what i have to follow to become fit


In [33]:
def final_response(query,token):
    print(query)
    remaining_token=225-token

    generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": remaining_token,
    "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config
    )

    response = model.generate_content(query)
    return response.text


response=final_response(combined_query[0],combined_query[1])
print(response)

  Can you tell what i have to follow to become fit
It's great you're looking to get fit! Here's a breakdown of what you need to focus on, but remember, **it's important to consult with a healthcare professional before starting any new exercise program.** 

**1. Set Realistic Goals**
* **Start small:** Don't try to do too much too soon. Aim for gradual progress.
* **Make it specific:** Instead of "lose weight," aim for "lose 1-2 pounds per week" or "walk 30 minutes 3 times a week."
* **Track your progress:** This will help you stay motivated and see how far you've come.

**2. Find Activities You Enjoy**
* **Variety is key:**  Don't be afraid to mix things up!  
* **Explore options:** Try different types of exercise: cardio (running, swimming, cycling), strength training (weights, bodyweight exercises), yoga, dance, etc.
* **Make it social:** Exercise with a friend or join
